In [12]:
# Raw Package
import numpy as np
import pandas as pd

#Data Source
import yfinance as yf

#Data viz
import plotly.graph_objs as go

stock_name = 'SOLARINDS.NS'

#Interval required 1 minute
data = yf.download(tickers= stock_name, period='1d', interval='1m')

#declare figure
fig = go.Figure()

#Candlestick
fig.add_trace(go.Candlestick(x=data.index,
                open=data['Open'],
                high=data['High'],
                low=data['Low'],
                close=data['Close'], name = 'market data'))

# Add titles
fig.update_layout(
    title=f'{ stock_name } live share price evolution',
    yaxis_title='Stock Price (USD per Shares)')

# # X-Axes
# fig.update_xaxes(
#     rangeslider_visible=True,
#     rangeselector=dict(
#         buttons=list([
#             dict(count=15, label="15m", step="minute", stepmode="backward"),
#             dict(count=45, label="45m", step="minute", stepmode="backward"),
#             dict(count=1, label="HTD", step="hour", stepmode="todate"),
#             dict(count=3, label="3h", step="hour", stepmode="backward"),
#             dict(step="all")
#         ])
#     )
# )

#Show
fig.show()

# Assuming 'data' contains the historical stock data fetched using yfinance
previous_candle = data.iloc[-2]  # Get the row for the previous candle

if previous_candle['Close'] > previous_candle['Open']:
    print("Previous candle was bullish.")
    # If bullish, buy 1 lot / stock at LIMIT with price as Close price of previous candle.
else:
    print("Previous candle was bearish.")
    # If bearish Sellshort 1 lot / stock at LIMIT with price as close price of previous candle

# 5) At the end of the candle (Say 59th second of a minute) close the position that are open(Long or Short) and 
#    delete all the unexecuted pending orders. (sometimes limit price not reached +, so orders may be pending)

[*********************100%***********************]  1 of 1 completed


Previous candle was bearish.
